# Automatic Dfferentiation

In [11]:
import torch

x = torch.arange(4.0)
x

tensor([0., 1., 2., 3.])

In [12]:
x.requires_grad_(True)
x.grad

In [13]:
y = 2 * torch.dot(x, x)
y

tensor(28., grad_fn=<MulBackward0>)

In [14]:
y.backward()
x.grad 

tensor([ 0.,  4.,  8., 12.])

In [15]:
x.grad == 4 * x

tensor([True, True, True, True])

 Instead, the new gradient is added to the already-stored gradient. This behavior comes in handy when we want to optimize the sum of multiple objective functions.

In [16]:
# PyTorch accumulates the gradient in default, we need to clear the previous
# values 
x.grad.zero_()
y = x.sum()
y.backward()
x.grad

tensor([1., 1., 1., 1.])

pytorch在利用计算图求导的过程中根节点都是一个标量，即一个数。当根节点即函数的因变量为一个向量的时候，会构建多个计算图对该向量中的每一个元素分别进行求导

向量对向量求导实际上是先计算y的雅可比矩阵，然后再将雅可比矩阵和元素为1形状和y相同的矩阵做点乘。

In [17]:
# Invoking `backward` on a non-scalar requires passing in a `gradient` argument
# which specifies the gradient of the differentiated function w.r.t `self`.
# In our case, we simply want to sum the partial derivatives, so passing
# in a gradient of ones is appropriate
x.grad.zero_()
y = x * x
# y.backward(torch.ones(len(x))) equivalent to the below
y.sum().backward()
x.grad

tensor([0., 2., 4., 6.])

Detachning computation.

In [19]:
x.grad.zero_()
y = x * x
u = y.detach()
z = u * x
z.sum().backward()
print(x.grad)
x.grad == u

tensor([0., 1., 4., 9.])


tensor([True, True, True, True])

In [20]:
x.grad.zero_()
y.sum().backward()
x.grad == 2 * x

tensor([True, True, True, True])

Computing the Gradient of Python Control Flow

In [21]:
def f(a):
    b = a * 2
    while b.norm() < 1000:
        b = b * 2
    if b.sum() > 0:
        c = b
    else:
        c = 100 * b
    return c

a = torch.randn(size=(), requires_grad=True)
d = f(a)
d.backward()

a.grad == d / a

tensor(True)